In [38]:
import sys
sys.path.append("../src/")
%load_ext autoreload
%autoreload 2

In [5]:
import sys
import csv

csv.field_size_limit(sys.maxsize)

131072

In [14]:
sents = 0
lens = []
with open("../../cross_domain_coherence/data/parsed_wsj/test_perm.tsv") as fd:
    rd = csv.reader(fd, delimiter="\t")
    for row in rd:
        example_len = len(row[1].split("<PUNC>"))
        lens.append(example_len)
        sents += example_len
print(sents)

22422


In [61]:
len(lens)

1054

In [60]:
for i in range(0, len(lens),  210):
    print(i)

0
210
420
630
840
1050


In [16]:
import numpy as np

In [17]:
np.mean(lens)

21.27324478178368

In [33]:
np.sum(np.array(lens) > 60) /  len(lens)

0.05218216318785579

# Details
* 22422 total sentences in 1054 examples
* average length of paragraph is 21 sentences
* max paragraph is 151 sentences
* We can probably fit ~60 sentences into our 512 window (we can eval in buckets of different seq_len sizes)
* only 5% of the data is outside our max window size

# TODO
* SRL encode all examples
* create data loader for this task
* evaluation
    * compute loss for each permutation, lowest loss should be for correct perm
    * bucket for different seq lengths?

In [86]:
import glob
import data
import torch
from torch.utils.data import DataLoader
import training

In [ ]:
args = """--emsize 768 --nhid 1536 --nlayers 2 --contrasts=10 --emsize 768 --nhid 1536 
--bptt 50 --batch_size 1 --val_data ../data/train/ --restore ../checkpoints/chkpt-1-2000-model.pt"""
runner = training.ModelRunner(args.split())

In [104]:
ds = data.InsertionDataset(glob.glob("../data/xu/*.pkl"), 128, torch.device("cpu"))

In [105]:
dl = DataLoader(ds, batch_size=1, num_workers=0)

In [115]:
runner.evaluate_insertion(dl, 10)

  0%|          | 3/1054 [00:12<1:20:23,  4.59s/it]

Batch size torch.Size([722, 1, 768])


  1%|          | 6/1054 [00:41<1:44:26,  5.98s/it]

Batch size torch.Size([519, 1, 768])


  1%|          | 9/1054 [00:56<1:16:50,  4.41s/it]

Batch size torch.Size([571, 1, 768])


  1%|          | 11/1054 [01:26<2:17:02,  7.88s/it]


0.4065573770491803

# Eval with gpt2

In [1]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel

In [2]:
# Load pre-trained model tokenizer (vocabulary)
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# Encode a text inputs
text = "Who was Jim Henson ? Jim Henson was a"
indexed_tokens = tokenizer.encode(text, return_tensors="pt")

In [18]:
# Load pre-trained model (weights)
model = GPT2LMHeadModel.from_pretrained('gpt2')

# Set the model in evaluation mode to deactivate the DropOut modules
# This is IMPORTANT to have reproducible results during evaluation!
model.eval()
pass

In [27]:
# Predict all tokens
with torch.no_grad():
    outputs = model(indexed_tokens, labels=indexed_tokens, return_dict=True)
    predictions = outputs[0]

In [28]:
outputs.loss

tensor(3.1810)

In [32]:
import csv
import sys
csv.field_size_limit(sys.maxsize)


131072

In [33]:
paragraphs = []
with  open("../../cross_domain_coherence/data/parsed_wsj/test_perm.tsv") as f:
    reader = csv.reader(f, delimiter='\t')
    for row in reader:
        paragraphs.append(row[1])

In [35]:
paragraphs = [p.split("<PUNC>") for p in paragraphs]

In [52]:
all_acc = []
for para in paragraphs:
    indexed_tokens = tokenizer.encode([" ".join(para)], return_tensors="pt")
    indexed_tokens = indexed_tokens[:,:512]
    with torch.no_grad():
        loss = model(indexed_tokens, labels=indexed_tokens, return_dict=True).loss
    true_loss = loss
    false_losses = []
    for i in range(len(para)):
        for j in range(len(para)):
            if i == j:
                continue
            moved_sent = para[i]
            other_sents = para[:i] + para[i+1:]
            before_sents = other_sents[:j]
            after_sents = other_sents[j:]
            new_order = before_sents + [moved_sent] + after_sents
            indexed_tokens = tokenizer.encode([" ".join(new_order)], return_tensors="pt")
            indexed_tokens = indexed_tokens[:,:512]
            with torch.no_grad():
                loss = model(indexed_tokens, labels=indexed_tokens, return_dict=True).loss
            print(loss)
            false_losses.append(loss)
    for loss in false_losses:
        all_acc.append(1 if loss > true_loss else 0)
    break



tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tens

KeyboardInterrupt: 

In [48]:
import transformers

In [55]:
indexed_tokens = tokenizer.encode(" ".join(para), return_tensors="pt")


In [57]:
indexed_tokens.shape

torch.Size([1, 1498])

In [58]:
len(" ".join(para).split(" "))

1331

In [39]:
import data_utils as du

In [40]:
paragraphs = du.get_xu_data("../../cross_domain_coherence/data/parsed_wsj/test_perm.tsv")


In [41]:
len(paragraphs)

1054

In [56]:
r = du.srl_paragraphs_batched_xu(paragraphs[:3], batch_size=16)

[{'verb': 'announced', 'description': 'When [ARG0: the Soviets] [V: announced] [ARG1: their last soldier had left Afghanistan in February] , the voices of skepticism were all but drowned out by an international chorus of euphoria .', 'tags': ['O', 'B-ARG0', 'I-ARG0', 'B-V', 'B-ARG1', 'I-ARG1', 'I-ARG1', 'I-ARG1', 'I-ARG1', 'I-ARG1', 'I-ARG1', 'I-ARG1', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']}, {'verb': 'had', 'description': 'When the Soviets announced their last soldier [V: had] left Afghanistan in February , the voices of skepticism were all but drowned out by an international chorus of euphoria .', 'tags': ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-V', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']}, {'verb': 'left', 'description': '[ARGM-TMP: When] the Soviets announced [ARG0: their last soldier] had [V: left] [ARG1: Afghanistan] [ARGM-TMP: in February] , the voices of skepticism were 

In [54]:
r[0][0]

{'verbs': [{'verb': 'announced',
   'description': 'When [ARG0: the Soviets] [V: announced] [ARG1: their last soldier had left Afghanistan in February] , the voices of skepticism were all but drowned out by an international chorus of euphoria .',
   'tags': ['O',
    'B-ARG0',
    'I-ARG0',
    'B-V',
    'B-ARG1',
    'I-ARG1',
    'I-ARG1',
    'I-ARG1',
    'I-ARG1',
    'I-ARG1',
    'I-ARG1',
    'I-ARG1',
    'O',
    'O',
    'O',
    'O',
    'O',
    'O',
    'O',
    'O',
    'O',
    'O',
    'O',
    'O',
    'O',
    'O',
    'O',
    'O',
    'O']},
  {'verb': 'had',
   'description': 'When the Soviets announced their last soldier [V: had] left Afghanistan in February , the voices of skepticism were all but drowned out by an international chorus of euphoria .',
   'tags': ['O',
    'O',
    'O',
    'O',
    'O',
    'O',
    'O',
    'B-V',
    'O',
    'O',
    'O',
    'O',
    'O',
    'O',
    'O',
    'O',
    'O',
    'O',
    'O',
    'O',
    'O',
    'O',
    'O

In [34]:
import pickle

In [57]:
f = open("../src/xu_ins_srl.pkl", 'rb')
unpickler = pickle.Unpickler(f)
x = unpickler.load()

In [58]:
x

[('When the Soviets announced their last soldier had left Afghanistan in February , the voices of skepticism were all but drowned out by an international chorus of euphoria .',
  [[('V', [4]), ('ARG0', [2, 3]), ('ARG1', [5, 6, 7, 8, 9, 10, 11, 12])],
   [('V', [8])],
   [('V', [9]), ('ARG0', [5, 6, 7]), ('ARG1', [10]), ('ARGM', [1, 11, 12])],
   [('V', [18])],
   [('V', [21]),
    ('ARG0', [23, 24, 25, 26, 27, 28, 29, 30]),
    ('ARG1', [14, 15, 16, 17]),
    ('ARGM', [19, 20])]]),
 ('It was " the Soviets \' Vietnam . " The Kabul regime would fall .',
  [[('V', [2]), ('ARG1', [1]), ('ARG2', [4, 5, 6, 7])],
   [('V', [13])],
   [('V', [14]), ('ARG1', [10, 11, 12]), ('ARGM', [13])]]),
 ('Millions of refugees would rush home .',
  [[('V', [4])],
   [('V', [5]), ('ARG1', [1, 2, 3]), ('ARG2', [6]), ('ARGM', [4])]]),
 ('A resistance government would walk into Kabul .',
  [[('V', [4])], [('V', [5]), ('ARG0', [1, 2, 3]), ('ARGM', [4, 6, 7])]]),
 ('Those who bought that illusion are now bewilde

In [62]:
tokenizer, _ = du.make_lm_encoder()

In [65]:
examples = ["hi, hello there",
           "hi, hello there. Thanks",
           "hi, hello there, you're welcome",
           "hi, hello there, wowza",
           "hi, hello there. no way!",
           "hi, hello there. ok fine but only because it's ok."]
out = tokenizer(examples, return_tensors="pt", padding=True,
                                         truncation=True, max_length=512)

In [79]:
examples[5:]

["hi, hello there. ok fine but only because it's ok."]